# Парафраз русских предложений с Hugging Face
Этот блокнот использует модель `cointegrated/rut5-base-paraphraser` для перефразирования русских текстов.
Также используется датасет `russian-superglue/ru_paraphraser` для проверки качества модели.

In [5]:
# Установка библиотек
# !pip install transformers datasets torch
!pip install transformers datasets

## 1. Загрузка датасета

In [8]:
from datasets import load_dataset

# Загружаем датасет
dataset = load_dataset("merionum/ru_paraphraser")

# Проверяем структуру
print(dataset)

README.md:   0%|          | 0.00/3.03k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/2.17M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/605k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7227 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1924 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})


In [10]:
dataset["train"][10]

{'id': '11',
 'id_1': '246',
 'id_2': '8165',
 'text_1': 'Москвичи смогут забронировать в Интернете место на кладбище.',
 'text_2': 'В Москве можно будет забронировать место на кладбище через интернет.',
 'class': '1'}

In [12]:
dataset["train"][0]

{'id': '1',
 'id_1': '201',
 'id_2': '8159',
 'text_1': 'Полицейским разрешат стрелять на поражение по гражданам с травматикой.',
 'text_2': 'Полиции могут разрешить стрелять по хулиганам с травматикой.',
 'class': '0'}

## 2. Фильтрация данных (только правильные парафразы)

In [15]:
# Фильтруем только примеры, где label = 1 (парафраз)
train_data = dataset["train"].filter(lambda x: x["class"] == '1')
test_data = dataset["test"].filter(lambda x: x["class"] == '1')

# Выводим пример
print(f"Количество примеров: {len(train_data)}")
print(train_data.shape)

Filter:   0%|          | 0/7227 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1924 [00:00<?, ? examples/s]

Количество примеров: 1688
(1688, 6)


## 3. Использование `pipeline` для парафраза

In [18]:
from transformers import pipeline

# Загружаем pipeline для парафраза
paraphraser = pipeline("text2text-generation", model="cointegrated/rut5-base-paraphraser")

# Пример работы модели
text = "Я собираюсь поехать в Москву завтра."
result = paraphraser(text, max_length=50, num_return_sequences=1)

print("Исходное:", text)
print("Перефразированное:", result[0]["generated_text"])

/opt/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use mps:0


Исходное: Я собираюсь поехать в Москву завтра.
Перефразированное: Я завтра поеду в Москву.


## 4. Проверка модели на тестовом датасете

In [20]:
correct = 0
total = 5  # Берем 5 примеров для теста

for i in range(total):
    original = test_data[i]["text_1"]
    expected = test_data[i]["text_2"]

    # Генерируем парафраз
    generated = paraphraser(original, max_length=50, num_return_sequences=1)[0]["generated_text"]

    print(f" Исходное: {original}")
    print(f"Ожидаемое: {expected}")
    print(f"Сгенерированное: {generated}")

 Исходное: Вертолет с 11 иностранцами на борту упал в Пакистане
Ожидаемое: В Пакистане упал вертолет с 11 иностранцами
Сгенерированное: В Пакистане упал вертолёт с иностранцами
 Исходное: Самолет вернулся в аэропорт Новосибирска из-за стука в салоне
Ожидаемое: Самолет вернулся в новосибирский аэропорт из-за таинственного стука
Сгенерированное: Самолет вернулся в Новосибирск из-за стука в салоне
 Исходное: Суд оправдал Васильеву в хищении акций на два миллиарда рублей
Ожидаемое: Суд оправдал Васильеву в хищении акций на 2 млрд рублей
Сгенерированное: Суд оправдал Васильеву за хищение акций на два миллиарда рублей
 Исходное: Пушков: у Обамы не хватило духа лично поздравить наш народ с Победой
Ожидаемое: Пушков: Обама не нашел в себе духа лично поздравить россиян с Победой
Сгенерированное: Пушков: Обамы не хватило духа поздравить народ с Победой
 Исходное: МЧС РФ: тела погибших российских дипломатов доставят из Непала 11 мая
Ожидаемое: Тела погибших в Непале российских дипломатов доставят

In [26]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# Инициализация объекта Rouge
rouge = Rouge()

for i in range(total):
    original = test_data[i]["text_1"]
    expected = test_data[i]["text_2"]

    # Генерация парафраза
    generated = paraphraser(original, max_length=50, num_return_sequences=1)[0]["generated_text"]

    print(f"Исходное: {original}")
    print(f"Ожидаемое: {expected}")
    print(f"Сгенерированное: {generated}")

    try:
        # Вычисление ROUGE-метрики
        scores = rouge.get_scores(generated, expected)[0]
        print(f"ROUGE-1: {scores['rouge-1']}")
        print(f"ROUGE-2: {scores['rouge-2']}")
        print(f"ROUGE-L: {scores['rouge-l']}")
    except Exception as e:
        print(f"Ошибка при вычислении ROUGE: {e}")

Исходное: Вертолет с 11 иностранцами на борту упал в Пакистане
Ожидаемое: В Пакистане упал вертолет с 11 иностранцами
Сгенерированное: В Пакистане упал вертолёт с иностранцами
ROUGE-1: {'r': 0.7142857142857143, 'p': 0.8333333333333334, 'f': 0.7692307642603551}
ROUGE-2: {'r': 0.3333333333333333, 'p': 0.4, 'f': 0.36363635867768596}
ROUGE-L: {'r': 0.7142857142857143, 'p': 0.8333333333333334, 'f': 0.7692307642603551}
Исходное: Самолет вернулся в аэропорт Новосибирска из-за стука в салоне
Ожидаемое: Самолет вернулся в новосибирский аэропорт из-за таинственного стука
Сгенерированное: Самолет вернулся в Новосибирск из-за стука в салоне
ROUGE-1: {'r': 0.625, 'p': 0.7142857142857143, 'f': 0.6666666616888889}
ROUGE-2: {'r': 0.2857142857142857, 'p': 0.2857142857142857, 'f': 0.2857142807142858}
ROUGE-L: {'r': 0.625, 'p': 0.7142857142857143, 'f': 0.6666666616888889}
Исходное: Суд оправдал Васильеву в хищении акций на два миллиарда рублей
Ожидаемое: Суд оправдал Васильеву в хищении акций на 2 млрд р

## Вывод
- Использовали датасет `ru_paraphraser` с Hugging Face.
- Фильтровали правильные примеры (class="1").
- Использовали `pipeline` для парафраза.
- Проверили качество модели на тестовых примерах.

Теперь можно **использовать код для генерации русских парафраз**!


# ЗАДАНИЕ
Допишите оценку модели по метрике ROUGE